# Práctica 4: Audio

In [49]:
import tkinter as tk
from tkinter import ttk, messagebox

import os
import sounddevice as sd

In [50]:
from processing.DataLoader import *
from processing.FFTCalculator import *
from processing.Visualizer import *

from processing.Signal import Signal

In [51]:
from processing.Filters import *

## Interfaz de usuario

In [52]:
tmin, tmax = 0, 1
peaks = np.array([])
amplitudes = np.array([])
filename = None

In [53]:
factory = DataLoaderFactory()
pitcher = PitchCalculator()

### Lógica de la interfaz

In [54]:
def get_mono(musicLoader, note):
    try:
        note = musicLoader.to_single_channel(note)
    except:
        pass
    return note

def on_select(event=None):
    global listbox, instrumento_seleccionado, tmin, tmax, peaks, amplitudes, signal, filename
    musicLoader = factory.create("music")
    visualizer = Visualizer(instrumento_seleccionado.get().lower())

    value = listbox.get(listbox.curselection())

    filename = f"notes/{instrumento_seleccionado.get().lower()}/{value}"
    sample_rate, note = musicLoader.load(filename)
    note = get_mono(musicLoader, note)

    signal = Signal(sample_rate, note, instrumento_seleccionado.get().lower(), filename)

    signal = visualizer.show_audio_signal_cv2(signal)
    signal = visualizer.show_fft_cv2(signal)

def modify_listbox():
    global listbox, frame_elementos, instrumento_seleccionado, signal
    listbox.delete(0, tk.END)

    directorio_imagenes = "./notes/" + instrumento_seleccionado.get().lower() + "/"
    lista_imagenes = [os.path.join(directorio_imagenes, f) for f in os.listdir(directorio_imagenes) if f.endswith(('.wav'))]
    for imagen in lista_imagenes:
        name = os.path.basename(imagen)
        listbox.insert(tk.END, name)


In [55]:
def show_transform():
    global listbox, instrumento_seleccionado, signal
    visualizer = Visualizer(instrumento_seleccionado.get().lower())
    signal = visualizer.show_fft_cv2(signal)

In [56]:
def generate_sine_wave(frequency, duration, sample_rate, amplitude):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    signal = 2*amplitude*np.sin(2 * np.pi * frequency * t)
    return signal


def sintetize():
    global listbox, instrumento_seleccionado, signal
    visualizer = Visualizer(instrumento_seleccionado.get().lower())
    calculator = FourierCalculator()

    visualizer.show_audio_signal_cv2(signal)

    _, yf, phase = calculator.get_fft(signal.note, signal.sample_rate)
    note = calculator.get_inverse_fft(yf, phase)
    reconstructed_signal = Signal(signal.sample_rate, note, signal.instrument, signal.filename)

    visualizer.show_audio_signal_cv2(reconstructed_signal)

    duration = 2
    time = np.linspace(0, 2, int(signal.sample_rate * duration))
    note = np.zeros(len(time))

    peaks, amplitudes, _ = signal.pitches(5, 50)

    max_amplitude = np.max(amplitudes)
    for peak, amplitude in zip(peaks, amplitudes):
        note += generate_sine_wave(peak, duration, signal.sample_rate, amplitude/max_amplitude)

    visualizer.show_audio_signal_cv2(Signal(signal.sample_rate, note, signal.instrument, None))

In [57]:
def save_audio():
    global listbox, name_text, instrumento_seleccionado, signal
    visualizer = Visualizer(instrumento_seleccionado.get().lower())
    signal = visualizer.show_audio_signal_cv2(signal)

    file = name_text.get(0.0, tk.END).strip()
    filename = f"notes/{instrumento_seleccionado.get().lower()}/{file}.wav"

    musicLoader = factory.create("music")
    musicLoader.save(filename, signal.get_signal(), signal.sample_rate)
    modify_listbox()

In [58]:
def record():
    global listbox, instrumento_seleccionado, name_text, duration
    sample_rate = 44100
    dur = int(duration.get(0.0, tk.END).strip())

    note = sd.rec(int(sample_rate * dur), samplerate=sample_rate, channels=1, dtype='float64')
    sd.wait()

    filename = f"notes/{instrumento_seleccionado.get().lower()}/{name_text.get(0.0, tk.END).strip()}.wav"
    musicLoader = factory.create("music")
    musicLoader.save(filename, note, sample_rate)
    modify_listbox()

In [59]:
def rename():
    global listbox, instrumento_seleccionado, name_text
    new_filename = f"notes/{instrumento_seleccionado.get().lower()}/{name_text.get(0.0, tk.END).strip()}.wav"
    old_filename = listbox.get(listbox.curselection())
    os.rename(f"notes/{instrumento_seleccionado.get().lower()}/{old_filename}", new_filename)
    print(f"Renamed {old_filename} to {new_filename}")
    modify_listbox()

In [60]:
def delete():
    global listbox, instrumento_seleccionado
    filename = listbox.get(listbox.curselection())
    os.remove(f"notes/{instrumento_seleccionado.get().lower()}/{filename}")
    modify_listbox()

In [61]:
def cargar_archivo():
    global signal, listbox, instrumento_seleccionado, filename
    musicLoader = factory.create("music")
    visualizer = Visualizer(instrumento_seleccionado.get().lower())

    filename = filedialog.askopenfilename(title="Selecciona un archivo de audio", filetypes=[("Archivos WAV", "*.wav")])
    sample_rate, note = musicLoader.load(filename)
    note = get_mono(musicLoader, note)

    signal = Signal(sample_rate, note, instrumento_seleccionado.get().lower(), filename)

    signal = visualizer.show_audio_signal_cv2(signal)
    signal = visualizer.show_fft_cv2(signal)


In [62]:
def generar_audio():
    global filename
    fs = 44100  # Frecuencia de muestreo
    duracion = 2.0  # Duración de la señal en segundos
    frecuencias = [5, 50, 120, 300]  # Frecuencias principales de la señal

    # Generar señal sinusoidal con las frecuencias definidas
    t = np.linspace(0, duracion, int(fs * duracion), endpoint=False)
    senal = sum(np.sin(2 * np.pi * f * t) for f in frecuencias)

    num_frecuencias_ruido = 15  # Número de frecuencias adicionales para el ruido
    for _ in range(num_frecuencias_ruido):
        frecuencia_ruido = np.random.uniform(10, fs / 2)
        fase_ruido = np.random.uniform(0, 2 * np.pi)
        senal += 0.2 * np.sin(2 * np.pi * frecuencia_ruido * t + fase_ruido)

    # Normalizar la señal para evitar saturación
    senal /= num_frecuencias_ruido

    # Agregar ruido blanco adicional
    ruido = np.random.normal(0, 0.5, t.shape)
    senal_con_ruido = senal + ruido

    FiltroDeSenales(t, senal_con_ruido, fs)

In [63]:
def apply_filters():
    global listbox, instrumento_seleccionado, filename

    if filename:
        fs, senal_original = read(filename)
        t = np.linspace(0, len(senal_original) / fs, num=len(senal_original))
        
        if len(senal_original.shape) > 1:
            senal_original = senal_original[:, 0]
        
        senal_original = senal_original / np.max(np.abs(senal_original))
        FiltroDeSenales(t, senal_original, fs)

    else:
        messagebox.showinfo("Filtro fallido", f"Seleccione un archivo audio o genere uno con ruido")

### Interfaz de usuario

In [64]:
ventana = tk.Tk()
ventana.title("Editor de imágenes")
ventana.geometry("500x660")
ventana.resizable(False, False)
ventana.grid_columnconfigure(0, weight=1)

frame_lista = tk.Frame(ventana)
frame_lista.grid(row=0, column=0, sticky="nsew", padx=10, pady=(10, 5))
frame_lista.grid_propagate(False) 

canvas = tk.Canvas(frame_lista, height=100)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

scrollbar = ttk.Scrollbar(frame_lista, orient="vertical", command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

frame_elementos = tk.Frame(canvas)
canvas.create_window((0, 0), window=frame_elementos, anchor="nw")

def on_frame_configure(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame_elementos.bind("<Configure>", on_frame_configure)

directorio_imagenes = "./notes/piano"
lista_imagenes = [os.path.join(directorio_imagenes, f) for f in os.listdir(directorio_imagenes) if f.endswith(('.wav'))]

listbox = tk.Listbox(frame_elementos, height=10, width=600)
for imagen in lista_imagenes:
    listbox.insert(tk.END, os.path.basename(imagen))
listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

listbox.bind("<<ListboxSelect>>", on_select)

canvas.configure(yscrollcommand=scrollbar.set)

frame_formulario = tk.Frame(ventana)
frame_formulario.grid(row=1, column=0, sticky="n", padx=10, pady=(10, 0))

tk.Button(frame_formulario, text="Cargar un Archivo", command=cargar_archivo).grid(row=0, column=0, columnspan=2, pady=5, sticky="wen")

tk.Label(frame_formulario, text="Seleccione un instrumento", font=("Arial", 12)).grid(row=1, column=0, columnspan=1, pady=10, sticky="w")

instrumentos = ["Guitarra", "Piano", "Violin", "Flauta", "Trompeta"]
instrumento_seleccionado = tk.StringVar(value="Piano")

for idx, instrumento in enumerate(instrumentos):
    tk.Radiobutton(
        frame_formulario, 
        text=instrumento, 
        variable=instrumento_seleccionado, 
        value=instrumento,
        command=modify_listbox
    ).grid(row=2+idx, column=1, columnspan=1, sticky="nw")

tk.Label(frame_formulario, text="Acciones", font=("Arial", 12)).grid(row=19, column=0, columnspan=1, pady=10, sticky="w")

tk.Button(frame_formulario, text="Reproducir", command=on_select).grid(row=20, column=0, columnspan=1, pady=5, padx=30, sticky="wen")
tk.Button(frame_formulario, text="Sintetizar", command=sintetize).grid(row=20, column=1, columnspan=1, pady=5, sticky="wne")
tk.Button(frame_formulario, text="Generar audio con ruido", command=generar_audio).grid(row=21, column=1, columnspan=1, pady=5, sticky="wne")

tk.Button(frame_formulario, text="Aplicar Filtros", command=apply_filters).grid(row=21, column=0, columnspan=1, pady=5, padx=30, sticky="wen")

tk.Label(frame_formulario, text="Nombre", font=("Arial", 10)).grid(row=22, column=0, columnspan=1, pady=10, sticky="w")
name_text = tk.Text(frame_formulario, height=1, width=30)
name_text.grid(row=23, column=0, columnspan=1, pady=5, padx=30, sticky="wen")
tk.Button(frame_formulario, text="Guardar", command=save_audio).grid(row=23, column=1, columnspan=1, pady=5, sticky="wen")
tk.Button(frame_formulario, text="Renombrar", command=rename).grid(row=24, column=1, columnspan=1, pady=5, sticky="wen")
tk.Button(frame_formulario, text="Eliminar", command=delete).grid(row=25, column=1, columnspan=1, pady=5, sticky="wen")

tk.Label(frame_formulario, text="Duración", font=("Arial", 10)).grid(row=25, column=0, columnspan=1, pady=10, sticky="w")
duration = tk.Text(frame_formulario, height=1, width=30)
duration.grid(row=26, column=0, columnspan=1, pady=5, padx=30, sticky="wen")
tk.Button(frame_formulario, text="Grabar", command=record).grid(row=26, column=1, columnspan=1, pady=5, sticky="wen")

frame_formulario.grid_columnconfigure(0, weight=1)
frame_formulario.grid_columnconfigure(1, weight=1)

ventana.mainloop()
